## Testing ipyrad on my data

**20170524**

I ran a branching assembly with 80%, 82%, and 85% similarity last week with the hopes of comparing the number of loci out the back end, as well as the Fis distribution and other metrics. While trying to do all of that this week, I talked with a collaborator who uses ipyrad and she realized that I didn't use the adapter filtering parameter and that this could influence my results. So, rerunning that assembly now. 